## 1. Параметры

1. **Параметры орбиты**

Одно из условий на периодичность. Давайте подберём такую высоту орбиты, чтобы 1 спутник приходил в ту же точку через какое-то количество оборотов.
И давайте максимизируем число этих оборотов. Зачем? Чтобы по возвращению спутника в стартовую точку, он замёл как можно больший путь.
Ведь чем больше он заметёт, тем точнее будет сетка. А так как мы минимизируем число спутников, то мы как раз этого и хотим.

Подбором рассчитаем нужные нам высоты. Давайте сделаем ограничение на минимальное число оборотов в 60, потому что мы так захотели.

In [41]:
from math import pi, sqrt

R = 6_371_302
MU = 398600.4415 * 10 ** 9
OMEGA_E = 7.29211 * 10 ** -5
T_E = 2 * pi / OMEGA_E

In [52]:
T_MAX = 100 * 3600

H_MIN = 400_000
H_MAX = 600_000
dH = 0.1


def print_periodic_h(resolution_max, k_min):
    h = H_MIN

    while h <= H_MAX:
        a = R + h
        T = 2 * pi * sqrt(a ** 3 / MU)
        delta_lambda = 360 * T / T_E

        k_max = int(T_MAX / T)
        for k in range(1, k_max + 1):
            phase = k * delta_lambda % 360
            delta = min(abs(phase), abs(360 - phase))

            resolution = delta / 360 * 2 * pi * R

            if resolution < resolution_max and k >= k_min:
                print(f'h = {h:.1f}; t = {k}; resolution = {resolution:.2f}')

        h += dH


print_periodic_h(100, 60)

h = 411187.7; t = 62; resolution = 97.30
h = 411187.8; t = 62; resolution = 93.76
h = 411187.9; t = 62; resolution = 90.22
h = 411188.0; t = 62; resolution = 86.68
h = 411188.1; t = 62; resolution = 83.13
h = 411188.2; t = 62; resolution = 79.59
h = 411188.3; t = 62; resolution = 76.05
h = 411188.4; t = 62; resolution = 72.51
h = 411188.5; t = 62; resolution = 68.97
h = 411188.6; t = 62; resolution = 65.43
h = 411188.7; t = 62; resolution = 61.89
h = 411188.8; t = 62; resolution = 58.34
h = 411188.9; t = 62; resolution = 54.80
h = 411189.0; t = 62; resolution = 51.26
h = 411189.1; t = 62; resolution = 47.72
h = 411189.2; t = 62; resolution = 44.18
h = 411189.3; t = 62; resolution = 40.64
h = 411189.4; t = 62; resolution = 37.10
h = 411189.5; t = 62; resolution = 33.56
h = 411189.6; t = 62; resolution = 30.01
h = 411189.7; t = 62; resolution = 26.47
h = 411189.8; t = 62; resolution = 22.93
h = 411189.9; t = 62; resolution = 19.39
h = 411190.0; t = 62; resolution = 15.85
h = 411190.1; t 

Возьмем высоту в $h=485115$ м. Для этой высоты периодичная орбита имеет максимальное число оборотов: 61. Мы отбросили $h=411190.4$ м с периодом в 62 оборота,
потому что на самом деле там период в 31 оборот. Период для нашей выбранной высоты будет $T \approx 2\pi\sqrt{a^3/\mu} \approx 92.6$ мин.
А смещение по экватору $\Delta\lambda_{\rm orbit} = 360^\circ\frac{T}{T_{\rm E}} \approx 23.61^\circ$.
И через 61 оборот спутник будет над точкой старта с разницей меньше метра. И это умещается в ограничение на периодичность в 100 часов.

А что по разрешению на экваторе? Раз спутник возвращается в ту же точку, то пересечения с экватором будет происходить через равные промежутки (без доказательства).
Так как восходящий и нисходящий трек считаются разными, то число пересечений будет $2 \cdot 61 = 122$ и разрешение составит $\approx 328.13$ км.
А это очень близко к поставленной планке в 200 км! Что-то нам подсказывает, что двух спутников в противофазе в одной плоскости будет достаточно для этой цели.
Ведь их траектории никогда не пересекутся.

In [65]:
H = 485115
ROTATIONS = 61

def print_params():
    a = R + H
    T = 2 * pi * sqrt(a ** 3 / MU)
    print(f'T = {T / 60:.1f} мин')

    delta_lambda = 360 * T / T_E
    print(f'delta_lambda = {delta_lambda:.2f}')

    phase = ROTATIONS * delta_lambda % 360
    phase_delta = min(phase, abs(360 - phase))
    error = phase_delta / 360 * 2 * pi * R
    print(f'error = {error:.2f} м')

    print(f'total_time = {ROTATIONS * T / 3600:.2f} ч')

    resolution = 360 / (ROTATIONS * 2)
    resolution = resolution / 360 * 2 * pi * R
    print(f'resolution = {resolution / 1000:.2f} км')

print_params()

T = 94.2 мин
delta_lambda = 23.61
error = 0.67 м
total_time = 95.74 ч
resolution = 328.13 км


Рассмотрим два спутника